In [ ]:
import plotly
import numpy as np

from numbers import Real

In [ ]:
def seller_commission(home_price: Real) -> float:
    """Calculate the seller commission on a home using the standard formula."""
    first_rate = 0.07
    second_rate = 0.025
    threshold = 100000
    return first_rate*min(threshold, home_price) + second_rate*max(0, home_price-threshold)

    
def buyer_commission(home_price: Real) -> float:
    """Calculate the buyer commission on a home using the standard formula."""
    first_rate = 0.03125
    second_rate = 0.011625
    threshold = 100000
    return first_rate*min(threshold, home_price) + second_rate*max(0, home_price-threshold)
    
def property_transfer_tax(home_price: Real) -> float:
    """Calculate the property transfer tax."""
    # for what jurisidiction?
    first_rate = 0.01
    second_rate = 0.02
    threshold = 200000
    return first_rate*min(threshold, home_price) + second_rate*max(0, home_price-threshold)

In [ ]:
# this cell is tagged 'parameters'

# basic parameters
home_price = 1500000
down_payment = 0.2
mortgage_rate = 0.03
amortization = 20

# speculative parameters

house_price_appreciation = 0.06
rental_appreciation = 0.02
return_on_investment = 0.04
inflation = 0.02

# costs

city = "vancouver"
# 2018 property tax rates 
tax_rates = {"vancouver": 0.00247, "toronto": 0.00636, "calgary": 0.00636, "saskatoon":0.0866, "edmonton":0.00882, "regina": 0.00949, "montreal": 0.00999, "quebec": 0.01011, "ottawa": 0.01068,"halifax":0.01206,"winnipeg":0.01212}
property_tax_rate = tax_rates[city] if city in tax_rates else 0.01
maintenance = 0.0025
strata_fee = 875 # 0.7% of value annually rule of thumb
utilities = 100
homeowner_insurance = 0.0046 # annual percentage of value

# renter costs

security_deposit = 0.5 # month's rent
renter_insurance = 20 # monthly

## Continuous Compounding

All rates will be converted to continuously compounding.

To convert from an annual interest rate, $r$, to a continuous rate, $r^*$:

$r* = ln(r + 1)$

This is easy because the period of the stated interest rate matches the compounding period. But sometimes annual rates are compounded at different intervals, eg monthly. In this case:

$r* = ln$

### Calculating the future value

Calculating the future value of an investment would normally be done as such:

$V = V_{0}(1 + r/n)^{nt}$

Where $n$ is the number of compounding periods and $r$ is the stated interest rate and $t$ is the time in interest rate periods.

With continuous rates, this simply reads:

$V=V_0e^{rt}$

### Calculating the Rate of Return

To caluclation the continuous rate of return, we use

$r_{log} = \frac{1}{t}\ln \big(\frac{V_f}{V_0}\big)$

In [ ]:
def continuous_rate(annual_rate: float, n_compounds: int = 1) -> float:
    """Convert an annual rate to a continuously compounding annual rate."""
    annual_return = (1+annual_rate/n_compounds)**(n_compounds)
    r_log = np.log(annual_return)
    return r_log


def mortgage_payment(mortgage: Real, mortgage_rate: float, term: int = 25) -> float:
    """
    Calculate the monthly mortgage payment.
    
    :param mortgage: The amount of the mortgage.
    :param mortgage_rate: The annual mortgage interest rate.
    :param term: The length of the mortgage in years.
    """
    r = mortgage_rate/12
    discount_factor = (r * (1+r)**(term*12))/((1+r)**(term*12)-1)
    mortgage_payment = mortgage*discount_factor
    return mortgage_payment

def opportunity_cost(initial_cost: float, recurring_costs: float, years: Real, return_on_investment: float):
    """Calculate the opportunity cost of owning the house."""
    roi = continuous_rate(annual_rate=return_on_investment, n_compounds=1)
    initial_opportunity = initial_cost * (np.exp(roi*years) - 1)
    recurring_opportunity = recurring_costs/(years*12) * ((np.exp(roi/12 * (years*12)) - 1) / (np.exp(roi/12) - 1) - years*12)
    return initial_opportunity + recurring_opportunity

In [ ]:
def buy (
    years: Real, 
    home_price: Real,
    appreciation:float, 
    return_on_investment: float, 
    mortgage_rate: float,
    maintenance: float,
    property_tax_rate: float,
    strata_fee: Real = 0,
    utilities: Real = 100,
    homeowner_insurance: float = 0.046,
    down_payment: float = 0.2, 
    inflation: float = 0.02,
    mortgage_term: int = 25,
    report: bool = False
):
    """
    Calculate the net financial cost of buying a house.
    
    :param years: The number of years spent in the house.
    :param home_price: The price of the home.
    :param appreciation: The estimated annual appreciation of the house.
    :param return_on_investment: The estimated annual return on investment from the market.
    :param mortgage_rate: The mortgage rate.
    :param maintenance: The cost of monthly maintenance, expressed as a fraction of the home value.
    :param property_tax_rate: The property tax rate.
    :param strata_fee: The monthly condo/strata fee. Default 0.
    :param utilities: The monthly cost of utilties.
    :param homeowner_insurance: The annual cost of insurance as a fraction of the home's value.
    :param down_payment: The fraction of the home price in down payment. Default 20%.
    :param inflation: The expected annual rate of inflation. Default is 2%.
    :param mortgage_term: The term of the mortgage. Default 25 years.
    :param report: Print the breakdown.
    """
    
    r = continuous_rate(annual_rate=appreciation, n_compounds=1)
    i = continuous_rate(annual_rate=inflation, n_compounds=1)
    
    # initial cost
    down_payment_cost = down_payment*home_price
    buyer_commission_cost = buyer_commission(home_price)
    property_tax_transfer_cost = property_transfer_tax(home_price)
    initial_costs = down_payment_cost + buyer_commission_cost + property_tax_transfer_cost
    
    # mortgage
    months = np.ceil(years*12)
    mortgage_amount = home_price*(1-down_payment)
    mortgage_payment_total = months * mortgage_payment(mortgage_amount, mortgage_rate, term=mortgage_term)
    mortgage_balance = mortgage_amount*(1 + mortgage_rate/12)**(months) - mortgage_payment_total
    
    # recurring costs
    property_tax_total = property_tax_rate * home_price * (1/r)*(np.exp(r*years)-np.exp(r*0))
    maintenance_total = maintenance * home_price * (1/r)*(np.exp(r*years)-np.exp(r*0))
    insurance_total = homeowner_insurance * 12 * (1/i)*(np.exp(i*years)-np.exp(i*0))
    utilities_total = utilities * 12 * (1/i)*(np.exp(i*years)-np.exp(i*0))
    strata_fee_total = strata_fee * 12 *  (1/i)*(np.exp(i*years)-np.exp(i*0))
        
    recurring_costs = property_tax_total + maintenance_total + insurance_total + utilities_total + strata_fee_total + mortgage_payment_total
    
    # opportunity costs
    opportunity_cost_total = opportunity_cost(initial_costs, recurring_costs, years, return_on_investment)

    final_home_price = home_price * np.exp(r*years)
    seller_commission_cost = seller_commission(final_home_price)
    proceeds = final_home_price - seller_commission_cost - mortgage_balance
    
    
    net_cost = initial_costs + recurring_costs + opportunity_cost_total - proceeds 
    
    if report:
        print("Initial Costs: ${0:,.0f}".format(initial_costs))
        print("  Down Payment: ${0:,.0f}".format(down_payment_cost))
        print("  Buyer Commission Cost: ${0:,.0f}".format(buyer_commission_cost))
        print("  Property Transfer Tax: ${0:,.0f}".format(property_tax_transfer_cost))
        print("Recurring Costs: ${0:,.0f}".format(recurring_costs))
        print("  Property Taxes: ${0:,.0f}".format(property_tax_total))
        print("  Maintenance: ${0:,.0f}".format(maintenance_total))
        print("  Insurance: ${0:,.0f}".format(insurance_total))
        print("  Utilities: ${0:,.0f}".format(utilities_total))
        print("  Strata Fees: ${0:,.0f}".format(strata_fee_total))
        print("  Mortgage Payments: ${0:,.0f}".format(mortgage_payment_total))
        print("Opportunity Costs: ${0:,.0f}".format(opportunity_cost_total))
        print("Proceeds: ${0:,.0f}".format(-proceeds))
        print("  Selling Price: ${0:,.0f}".format(-final_home_price))
        print("  Seller Commission: ${0:,.0f}".format(seller_commission_cost))
        print("  Mortgage Balance: ${0:,.0f}".format(mortgage_balance))
        print("Net Cost: ${0:,.0f}".format(net_cost))
    
    return net_cost    

In [ ]:
net_financial_position = buy(
    years=5,
    home_price=home_price,
    appreciation=house_price_appreciation,
    return_on_investment=return_on_investment,
    mortgage_rate=mortgage_rate,
    maintenance=maintenance,
    property_tax_rate=property_tax_rate,
    strata_fee=strata_fee,
    utilities=utilities,
    homeowner_insurance=homeowner_insurance,
    down_payment=down_payment,
    inflation=inflation,
    report=True
)